In [1]:
import sys
sys.path.append('/home/xinyiz/pamrats')

import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim

# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import average_precision_score

import gae.gae.optimizer as optimizer
import gae.gae.model
import gae.gae.preprocessing as preprocessing

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import umap
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN,MiniBatchKMeans,AgglomerativeClustering
from sklearn import metrics

import pseudotime.fgw as fgw

import anndata as ad
import gc

In [13]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
ifplot=True

otpairs={'c13d13':['control13','disease13'],'c8c13':['control8','control13'],'d8d13':['disease8','disease13'],'c8d8':['control8','control13']}
# otpairs={'c8c13':['control8','control13']}
fgw_alpha=0.00001
costfn='euclidean'
# costfn='kl'

inverseAct='leakyRelu'
# inverseAct=None
plottype='umap'
pca=PCA()
minCells=15 #min number of cells for analysis
# clustermethod=['kmeanbatch']
clustermethod=['leiden','dbscan','agglomerative','kmeanbatch']
#umap/leiden clustering parameters
n_neighbors=10
min_dist=0.25
n_pcs=40 #for clustering
# resolution=[0.5,1,1.5,2]
resolution=[0.05,0.1,0.2,0.3]
plotepoch=9990
savenameAdd=''
#DBscan
epslist= [6,8,10]
min_sampleslist=[15,30,45] 
#agglomerative
nclusterlist=[2,3,4,5,8,10]
aggMetric=['euclidean']

combineCelltype={'glia':['Astro','Micro', 'OPC', 'Oligo'],'CA':['CA1', 'CA2', 'CA3']}

use_cuda=True
fastmode=False #Validate during training pass
seed=3
useSavedMaskedEdges=False
maskedgeName='knn20_connectivity'
hidden1=1024 #Number of units in hidden layer 1
hidden2=1024 #Number of units in hidden layer 2
# hidden3=16
fc_dim1=1024
# fc_dim2=2112
# fc_dim3=2112
# fc_dim4=2112
# gcn_dim1=2048

protein=None #'scaled_binary'
# proteinWeights=0.05
dropout=0.01
# randFeatureSubset=None
model_str='gcn_vae_xa_e2_d1_dca'
adj_decodeName=None #gala or None
plot_samples={'disease13':'AD_mouse9494','control13':'AD_mouse9498','disease8':'AD_mouse9723','control8':'AD_mouse9735'}
# plot_sample_X=['logminmax']
plot_sample_X=['raw']

muplotScale='minmax'
plotRecon='' #'meanRecon'
# plot_sample_X=['corrected','scaled']
standardizeX=False
name='allk20XA_01_dca'
logsavepath='/mnt/xinyi/pamrats/log/train_gae_starmap/'+name
modelsavepath='/mnt/xinyi/pamrats/models/train_gae_starmap/'+name
plotsavepath='/mnt/xinyi/pamrats/plots/train_gae_starmap/'+name
    

In [3]:
# Set cuda and seed
np.random.seed(seed)
if use_cuda and (not torch.cuda.is_available()):
    print('cuda not available')
    use_cuda=False
torch.manual_seed(seed)
if use_cuda:
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.enabled = True


In [4]:
def filter_genes(exprs, threshold = 15):
    """
    Removes columns (genes) with less than 'threshold' non-zero entries.
    """
    nonzero_col_sum =np.sum(exprs>0,axis=0)
    exprs = exprs[:, nonzero_col_sum>=threshold]

    return exprs

In [5]:
#Load data
savedir=os.path.join('/mnt/xinyi/','starmap')
adj_dir=os.path.join(savedir,'a')

featureslist={}
if plot_sample_X[0] in ['corrected','scaled']:
    scaleddata=scanpy.read_h5ad('/mnt/xinyi/2021-01-13-mAD-test-dataset/2020-12-27-starmap-mAD-scaled.h5ad')
    
    for s in plot_samples.keys():
        featureslist[s+'X_'+'corrected']=torch.tensor(scaleddata.layers['corrected'][scaleddata.obs['sample']==plot_samples[s]])
        featureslist[s+'X_'+'scaled']=torch.tensor(scaleddata.layers['scaled'][scaleddata.obs['sample']==plot_samples[s]])
    
else:
    scaleddata=scanpy.read_h5ad('/mnt/xinyi/2021-01-13-mAD-test-dataset/2020-12-27-starmap-mAD-raw.h5ad')
    
    for s in plot_samples.keys():
        scaleddata_train=scaleddata.X[scaleddata.obs['sample']==plot_samples[s]]
        scaleddata_train=filter_genes(scaleddata_train)
        
        if plot_sample_X[0]=='logminmax':
            featurelog_train=np.log2(scaleddata_train+1/2)
            scaler = MinMaxScaler()
            featurelog_train_minmax=np.transpose(scaler.fit_transform(np.transpose(featurelog_train)))
            featureslist[s+'X_'+plot_sample_X[0]]=torch.tensor(featurelog_train_minmax)
        if plot_sample_X[0]=='raw':
            featureslist[s+'X_'+plot_sample_X[0]]=torch.tensor(scaleddata_train)

if protein: ##adjust for scaled/corrected
    proteinsavepath=os.path.join('/mnt/xinyi/','starmap','protein')
    for s in plot_samples.keys():
        pmtx=sp.load_npz(os.path.join(proteinsavepath,plot_samples[s]+'_'+protein+'.npz'))
        pmtx=preprocessing.sparse_mx_to_torch_sparse_tensor(pmtx)
        pmtx=pmtx.to_dense()
        scalefactor=torch.sum(featureslist[s+'X_'+plot_sample_X[0]])/torch.sum(pmtx)*proteinWeights
        featureslist[s+'X_'+plot_sample_X[0]]=torch.cat((featureslist[s+'X_'+plot_sample_X[0]],pmtx*scalefactor),dim=1)

adj_list={}
adj_list['disease13']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9494.npz'))
adj_list['control13']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9498.npz'))
adj_list['disease8']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9723.npz'))
adj_list['control8']=sp.load_npz(os.path.join(adj_dir,maskedgeName+'_AD_mouse9735.npz'))

In [6]:
# load model
num_nodes,num_features = list(featureslist.values())[0].shape
if model_str=='gcn_vae_xa':
    model  = gae.gae.model.GCNModelVAE_XA(num_features, hidden1, hidden2,fc_dim1,fc_dim2,fc_dim3,fc_dim4, dropout)
elif model_str=='fc1':
    model  = gae.gae.model.FCVAE1(num_features, hidden1,dropout)
elif model_str == 'gcn_vae_xa_e2_d1':
    model  = gae.gae.model.GCNModelVAE_XA_e2_d1(num_features, hidden1,hidden2, dropout)
elif model_str == 'gcn_vae_gcnX_inprA':
    model = gae.gae.model.GCNModelVAE_gcnX_inprA(num_features, hidden1, hidden2,gcn_dim1, dropout)
elif model_str=='fc1_dca':
    model = gae.gae.model.FCVAE1_DCA(num_features, hidden1,fc_dim1, dropout)
elif model_str=='gcn_vae_xa_e2_d1_dca':
    model = gae.gae.model.GCNModelVAE_XA_e2_d1_DCA(num_features, hidden1,hidden2,fc_dim1, dropout)
elif model_str=='gcn_vae_xa_e2_d1_dcaFork':
    model = gae.gae.model.GCNModelVAE_XA_e2_d1_DCAfork(num_features, hidden1,hidden2,fc_dim1, dropout)
elif model_str=='gcn_vae_xa_e2_d1_dcaElemPi':
    model = gae.gae.model.GCNModelVAE_XA_e2_d1_DCAelemPi(num_features, hidden1,hidden2,fc_dim1, dropout,shareGenePi)
elif model_str=='gcn_vae_xa_e2_d1_dcaConstantDisp':
    model = gae.gae.model.GCNModelVAE_XA_e2_d1_DCA_constantDisp(num_features, hidden1,hidden2,fc_dim1, dropout,shareGenePi)
else:
    print('model not found')
model.load_state_dict(torch.load(os.path.join(modelsavepath,str(plotepoch)+'.pt')))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
def plotCoupling(cp,label1,label2,savepath):
    rownames=np.unique(label1)
    colnames=np.unique(label2)
    res=np.zeros((rownames.size,colnames.size))
    for l1 in range(rownames.size):
        l1Label=rownames[l1]
        l1Idx=(label1==l1Label)
        for l2 in range(colnames.size):
            l2Label=colnames[l2]
            l2Idx=(label2==l2Label)
            submtx=cp[l1Idx]
            submtx=submtx[:,l2Idx]
            res[l1,l2]=np.sum(submtx)/(np.sum(l1Idx)*np.sum(l2Idx))
            
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(res,cmap='binary')
    ax.set_yticks(np.arange(rownames.size))
    ax.set_yticklabels(rownames)
    ax.set_xticks(np.arange(colnames.size))
    ax.set_xticklabels(colnames)
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor")
    fig.tight_layout()
    plt.savefig(savepath)
    plt.close()

In [8]:
np.random.seed(seed)
def inverseLeakyRelu(v,slope=0.01):
    vnegidx=(v<0)
    v[vnegidx]=1/slope*v[vnegidx]
    return v

In [9]:
#compute embeddings
mulist={}
for s in plot_samples.keys():
    adj=adj_list[s]
    adj_norm = preprocessing.preprocess_graph(adj)
    adj_decode=None
    if adj_decodeName == 'gala':
        adj_decode=preprocessing.preprocess_graph_sharp(adj)
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        features=featureslist[samplename]
        if standardizeX:
            features=torch.tensor(scale(features,axis=0, with_mean=True, with_std=True, copy=True))
        if use_cuda:
            model.cuda()
            features = features.cuda().float()
            adj_norm=adj_norm.cuda()
            if adj_decodeName:
                adj_decode=adj_decode.cuda()
        
        model.eval()
        if adj_decodeName==None:
            adj_recon,mu,logvar,z, features_recon = model(features, adj_norm)
        else:
            adj_recon,mu,logvar,z, features_recon = model(features, adj_norm,adj_decode)
        if inverseAct=='leakyRelu':
            muplot=inverseLeakyRelu(mu.cpu().detach().numpy())
        else:
            muplot=mu.cpu().detach().numpy()
            
        if plotRecon:
            if plotRecon=='meanRecon':
                mulist[samplename]=features_recon[3].cpu().detach().numpy()
        else:
            mulist[samplename]=muplot
        
        if muplotScale=='minmax':
            scaler = MinMaxScaler()
            mulist[samplename]=np.transpose(scaler.fit_transform(np.transpose(mulist[samplename])))
features=features.cpu()        
adj_norm=adj_norm.cpu()

In [14]:
#all cells - compute OT
# ifcluster=False
np.random.seed(seed)
for spairs in otpairs.keys():
    print(spairs)
    s1,s2=otpairs[spairs]
    sampleidx1=plot_samples[s1]
    sampleidx2=plot_samples[s2]
    
    celltype_broad1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'top_level']
    celltype_sub1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'cell_type_label']
    region1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'region']
    sobj_coord_np1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,['x','y']].to_numpy()
    
    celltype_broad2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'top_level']
    celltype_sub2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'cell_type_label']
    region2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'region']
    sobj_coord_np2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,['x','y']].to_numpy()
    for xcorr in plot_sample_X:
        samplename1=s1+'X_'+xcorr
        muplot1=np.copy(mulist[samplename1])
        feature1=featureslist[samplename1].numpy()
        samplename2=s2+'X_'+xcorr
        muplot2=np.copy(mulist[samplename2])
        feature2=featureslist[samplename2].numpy()
        
        if inverseAct:
            samplename1+='_beforeAct'
            samplename2+='_beforeAct'
       
        sampledir=os.path.join(plotsavepath,spairs+plotRecon)
        savedir_emb=os.path.join(sampledir,'embedding')
        savedir_f=os.path.join(sampledir,'feature')
        if not os.path.exists(sampledir):
            os.mkdir(sampledir)
        if not os.path.exists(savedir_emb):
            os.mkdir(savedir_emb)
        if not os.path.exists(savedir_f):
            os.mkdir(savedir_f)
            
        fgw_feature,log_feature=fgw.getFGWcoupling(feature1,sobj_coord_np1,feature2,sobj_coord_np2,fgw_alpha,costfn,maxiter=1000000)
        fgw_embedding,log_embedding=fgw.getFGWcoupling(muplot1,sobj_coord_np1,muplot2,sobj_coord_np2,fgw_alpha,costfn,maxiter=1000000)
        
        with open(os.path.join(savedir_emb,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(fgw_embedding, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(savedir_emb,'log_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(log_embedding, output, pickle.HIGHEST_PROTOCOL)
            
        with open(os.path.join(savedir_f,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(fgw_feature, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(savedir_f,'log_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(log_feature, output, pickle.HIGHEST_PROTOCOL)
       


c13d13
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|8.243134e+01|0.000000e+00|0.000000e+00
    1|6.393827e+01|2.892333e-01|1.849308e+01
    2|2.624325e+01|1.436370e+00|3.769502e+01
    3|2.531741e+01|3.656927e-02|9.258393e-01
    4|2.531142e+01|2.368219e-04|5.994297e-03
    5|2.531079e+01|2.459217e-05|6.224475e-04
    6|2.531061e+01|7.360343e-06|1.862948e-04
It.  |Loss        |Relative loss|Absolute loss
------------------------------------------------
    0|6.072469e+01|0.000000e+00|0.000000e+00
    1|3.697387e+01|6.423677e-01|2.375082e+01
    2|5.920938e+00|5.244596e+00|3.105293e+01
    3|5.764711e+00|2.710067e-02|1.562275e-01
    4|5.752910e+00|2.051308e-03|1.180099e-02
    5|5.748515e+00|7.644103e-04|4.394224e-03
    6|5.746810e+00|2.967716e-04|1.705490e-03
    7|5.746170e+00|1.114322e-04|6.403084e-04
    8|5.745960e+00|3.645991e-05|2.094972e-04
    9|5.745878e+00|1.426745e-05|8.197906e-05
   10|5.745842e+00|6.371861e-06|3.66

In [15]:
#all cells - plot
# ifcluster=False
np.random.seed(seed)
for spairs in otpairs.keys():
    print(spairs)
    s1,s2=otpairs[spairs]
    sampleidx1=plot_samples[s1]
    sampleidx2=plot_samples[s2]
    
    celltype_broad1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'top_level']
    celltype_sub1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'cell_type_label']
    region1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'region']
    sobj_coord_np1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,['x','y']].to_numpy()
    
    celltype_broad2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'top_level']
    celltype_sub2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'cell_type_label']
    region2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'region']
    sobj_coord_np2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,['x','y']].to_numpy()
    for xcorr in plot_sample_X:
        samplename1=s1+'X_'+xcorr
        samplename2=s2+'X_'+xcorr
        
        if inverseAct:
            samplename1+='_beforeAct'
            samplename2+='_beforeAct'
       
        sampledir=os.path.join(plotsavepath,spairs+plotRecon)
        savedir_emb=os.path.join(sampledir,'embedding')
        savedir_f=os.path.join(sampledir,'feature')
        savedir_embplot=os.path.join(sampledir,'embedding','plots')
        savedir_fplot=os.path.join(sampledir,'feature','plots')
        if not os.path.exists(savedir_embplot):
            os.mkdir(savedir_embplot)
        if not os.path.exists(savedir_fplot):
            os.mkdir(savedir_fplot)
        
        with open(os.path.join(savedir_emb,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'rb') as input:
            fgw_embedding=pickle.load(input)
            
        with open(os.path.join(savedir_f,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'rb') as input:
            fgw_feature=pickle.load(input)
       
        plotCoupling(fgw_feature,celltype_broad1,celltype_broad2,os.path.join(savedir_fplot,'celltype_broad'+'_pi_alpha'+str(fgw_alpha)+'_'+costfn+'.jpg'))
        plotCoupling(fgw_embedding,celltype_broad1,celltype_broad2,os.path.join(savedir_embplot,'celltype_broad'+'_pi_alpha'+str(fgw_alpha)+'_'+costfn+'.jpg'))


c13d13
c8c13
d8d13
c8d8


In [13]:
# microglia
#all cells - compute OT
ct='Micro'
np.random.seed(seed)
for spairs in otpairs.keys():
    print(spairs)
    s1,s2=otpairs[spairs]
    sampleidx1=plot_samples[s1]
    sampleidx2=plot_samples[s2]
    
    celltype_broad1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'top_level']
    celltype_sub1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'cell_type_label']
    region1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'region']
    sobj_coord_np1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,['x','y']].to_numpy()
    
    celltype_broad2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'top_level']
    celltype_sub2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'cell_type_label']
    region2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'region']
    sobj_coord_np2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,['x','y']].to_numpy()
    
    origCT=np.unique(celltype_broad1)
    celltypeplot=np.concatenate((origCT,list(combineCelltype.keys())),axis=None)
    for xcorr in plot_sample_X:
        samplename1=s1+'X_'+xcorr
        muplot1=np.copy(mulist[samplename1])
        feature1=featureslist[samplename1].numpy()
        samplename2=s2+'X_'+xcorr
        muplot2=np.copy(mulist[samplename2])
        feature2=featureslist[samplename2].numpy()
        
        if inverseAct:
            samplename1+='_beforeAct'
            samplename2+='_beforeAct'
       
        sampledir=os.path.join(plotsavepath,spairs+plotRecon)
        savedir_emb=os.path.join(sampledir,'embedding'+ct)
        savedir_f=os.path.join(sampledir,'feature'+ct)
        if not os.path.exists(sampledir):
            os.mkdir(sampledir)
        if not os.path.exists(savedir_emb):
            os.mkdir(savedir_emb)
        if not os.path.exists(savedir_f):
            os.mkdir(savedir_f)
            
        if ct in origCT:
            ct_idx1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'top_level']==ct
            ct_idx2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'top_level']==ct
        else:
            ct_idx=False
            for i in combineCelltype[ct]:
                ct_idx=np.logical_or(ct_idx,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']==i)

            
        fgw_feature,log_feature=fgw.getFGWcoupling(feature1[ct_idx1],sobj_coord_np1[ct_idx1],feature2[ct_idx2],sobj_coord_np2[ct_idx2],fgw_alpha,costfn,maxiter=1000000)
        fgw_embedding,log_embedding=fgw.getFGWcoupling(muplot1[ct_idx1],sobj_coord_np1[ct_idx1],muplot2[ct_idx2],sobj_coord_np2[ct_idx2],fgw_alpha,costfn,maxiter=1000000)
        
        with open(os.path.join(savedir_emb,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(fgw_embedding, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(savedir_emb,'log_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(log_embedding, output, pickle.HIGHEST_PROTOCOL)
            
        with open(os.path.join(savedir_f,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(fgw_feature, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(savedir_f,'log_alpha'+str(fgw_alpha)+'_'+costfn), 'wb') as output:
            pickle.dump(log_feature, output, pickle.HIGHEST_PROTOCOL)
       


c13d13


NameError: name 'celltype_broad' is not defined

In [ ]:
#all cells - plot
# ifcluster=False
np.random.seed(seed)
for spairs in otpairs.keys():
    print(spairs)
    s1,s2=otpairs[spairs]
    sampleidx1=plot_samples[s1]
    sampleidx2=plot_samples[s2]
    
    celltype_broad1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'top_level']
    celltype_sub1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'cell_type_label']
    region1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'region']
    sobj_coord_np1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,['x','y']].to_numpy()
    
    celltype_broad2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'top_level']
    celltype_sub2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'cell_type_label']
    region2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'region']
    sobj_coord_np2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,['x','y']].to_numpy()
    for xcorr in plot_sample_X:
        samplename1=s1+'X_'+xcorr
        samplename2=s2+'X_'+xcorr
        
        if inverseAct:
            samplename1+='_beforeAct'
            samplename2+='_beforeAct'
       
        sampledir=os.path.join(plotsavepath,spairs+plotRecon)
        savedir_emb=os.path.join(sampledir,'embedding'+ct)
        savedir_f=os.path.join(sampledir,'feature'+ct)
        savedir_embplot=os.path.join(savedir_emb,'plots')
        savedir_fplot=os.path.join(savedir_f,'plots')
        if not os.path.exists(savedir_embplot):
            os.mkdir(savedir_embplot)
        if not os.path.exists(savedir_fplot):
            os.mkdir(savedir_fplot)
        
        with open(os.path.join(savedir_emb,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'rb') as input:
            fgw_embedding=pickle.load(input)
            
        with open(os.path.join(savedir_f,'pi_alpha'+str(fgw_alpha)+'_'+costfn), 'rb') as input:
            fgw_feature=pickle.load(input)
       
    
        if ct in origCT:
            ct_idx1=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx1,'top_level']==ct
            ct_idx2=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx2,'top_level']==ct
        else:
            ct_idx=False
            for i in combineCelltype[ct]:
                ct_idx=np.logical_or(ct_idx,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']==i)

        plotCoupling(fgw_feature,celltype_sub1[ct_idx1],celltype_sub2[ct_idx2],os.path.join(savedir_fplot,'celltype_sub'+'_pi_alpha'+str(fgw_alpha)+'_'+costfn+'.jpg'))
        plotCoupling(fgw_embedding,celltype_sub1[ct_idx1],celltype_sub2[ct_idx2],os.path.join(savedir_embplot,'celltype_sub'+'_pi_alpha'+str(fgw_alpha)+'_'+costfn+'.jpg'))


In [ ]:
# separate plots by cell type
np.random.seed(seed)
for s in plot_samples.keys():
#     if s in ['disease13','control13','disease8']:
#         continue
    print(s)
    sampleidx=plot_samples[s]
    celltype_broad=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']
    celltype_sub=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'cell_type_label']
    region=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'region']
    sobj_coord_np=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,['x','y']].to_numpy()
    
    origCT=np.unique(celltype_broad)
    celltypeplot=np.concatenate((origCT,list(combineCelltype.keys())),axis=None)
    for xcorr in plot_sample_X:
        samplename=s+'X_'+xcorr
        muplot=np.copy(mulist[samplename])
        
        if inverseAct:
            samplename+='_beforeAct'
        sampledir=os.path.join(plotsavepath,samplename)
        if not os.path.exists(sampledir):
            os.mkdir(sampledir)
        
        for ct in celltypeplot:
#             if s=='disease8' and ct in ['Astro','CA1','CA2','CA3','DG','Endo','Ex','Inhi']:
#             if s=='control8' and ct in ['Astro']:
#                 continue
            print(ct)
            
            savedir=os.path.join(plotsavepath,samplename,'embedding_'+plottype+'_'+ct)
            clustersavedir=os.path.join(plotsavepath,samplename,'cluster'+'_'+ct)
            if not os.path.exists(savedir):
                os.mkdir(savedir)
            if not os.path.exists(clustersavedir):
                os.mkdir(clustersavedir)
            
            if ct in origCT:
                ct_idx=scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']==ct
            else:
                ct_idx=False
                for i in combineCelltype[ct]:
                    ct_idx=np.logical_or(ct_idx,scaleddata.obs.loc[scaleddata.obs['sample']==sampleidx,'top_level']==i)
            
            if plottype=='umap':
                reducer = umap.UMAP(n_neighbors=n_neighbors,min_dist=min_dist,random_state=seed)
                embedding = reducer.fit_transform(muplot[ct_idx])
                savenameAdd='_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'epoch'+str(plotepoch)
            elif plottype=='pca':
                embedding=pca.fit_transform(muplot[ct_idx])
                savenameAdd='_epoch'+str(plotepoch)
                
            if ifplot:
    #             plotembeddingbyCT(celltype_broad[reg_idx],'celltype_broad',[],embedding,savedir,plottype+' of '+s+' '+reg)
                plotembeddingbyCT(celltype_sub[ct_idx],'celltype_sub',[],embedding,savedir,plottype+' of '+s+' '+ct,savenameAdd=savenameAdd)
                plotembeddingbyCT(region[ct_idx],'region',[],embedding,savedir,plottype+' of '+s+' '+ct,savenameAdd=savenameAdd)

                plotembeddingbyCT_contrast(celltype_sub[ct_idx],'celltype_sub',[],embedding,os.path.join(savedir,'contrast'),plottype+' of '+s+' '+ct,savenameAdd=savenameAdd)
            
            if embedding.shape[0]<minCells:
                continue
            if ifcluster:
                if 'leiden' in clustermethod:
                    clusterLeiden(muplot[ct_idx],n_neighbors,n_pcs,min_dist,resolution,sobj_coord_np[ct_idx],randseed=seed)
                    assert np.sum(muplot-np.copy(mulist[s+'X_'+xcorr]))==0
                if 'dbscan' in clustermethod:
                    clusterDBscan(muplot[ct_idx],epslist,min_sampleslist,n_pcs,sobj_coord_np[ct_idx])
                    assert np.sum(muplot-np.copy(mulist[s+'X_'+xcorr]))==0
                if 'agglomerative' in clustermethod:
                    clusterAgg(muplot[ct_idx],nclusterlist,aggMetric,n_pcs,sobj_coord_np[ct_idx])
                    assert np.sum(muplot-np.copy(mulist[s+'X_'+xcorr]))==0
                if 'kmeanbatch' in clustermethod:
                    clusterMinibatchKmean(muplot[ct_idx],nclusterlist,n_pcs,sobj_coord_np[ct_idx])
                    assert np.sum(muplot-np.copy(mulist[s+'X_'+xcorr]))==0

In [ ]:
s